**The problem**<br>


1.   What is our input?

2.   What is our output?

3.   What learning model?



In [1]:
# Case5.m :

# MATPOWER Version 7.0, 20-Jun-2019 -- AC Power Flow (Newton)
# 
# Newton's method power flow (power balance, polar) converged in 3 iterations.
# 
# Converged in 0.59 seconds
# ================================================================================
# |     System Summary                                                           |
# ================================================================================
# 
# How many?                How much?              P (MW)            Q (MVAr)
# ---------------------    -------------------  -------------  -----------------
# Buses              5     Total Gen Capacity    1530.0       -1147.5 to 1147.5
# Generators         5     On-line Capacity      1530.0       -1147.5 to 1147.5
# Committed Gens     5     Generation (actual)   1005.0             371.3
# Loads              3     Load                  1000.0             328.7
#   Fixed            3       Fixed               1000.0             328.7
#   Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
# Shunts             0     Shunt (inj)             -0.0               0.0
# Branches           6     Losses (I^2 * Z)         5.03             50.27
# Transformers       0     Branch Charging (inj)     -                7.7
# Inter-ties         0     Total Inter-tie Flow     0.0               0.0
# Areas              1
# 
#                           Minimum                      Maximum
#                  -------------------------  --------------------------------
# Voltage Magnitude   0.989 p.u. @ bus 2          1.000 p.u. @ bus 4   
# Voltage Angle      -0.76 deg   @ bus 2          4.11 deg   @ bus 5   
# P Losses (I^2*R)             -                  1.77 MW    @ line 1-2
# Q Losses (I^2*X)             -                 17.67 MVAr  @ line 1-2
# 
# ================================================================================
# |     Bus Data                                                                 |
# ================================================================================
#  Bus      Voltage          Generation             Load        
#   #   Mag(pu) Ang(deg)   P (MW)   Q (MVAr)   P (MW)   Q (MVAr)
# ----- ------- --------  --------  --------  --------  --------
#     1  1.000    3.273    210.00     30.73       -         -   
#     2  0.989   -0.759       -         -      300.00     98.61 
#     3  1.000   -0.492    323.49    194.65    300.00     98.61 
#     4  1.000    0.000*     5.03    184.12    400.00    131.47 
#     5  1.000    4.112    466.51    -38.21       -         -   
#                         --------  --------  --------  --------
#                Total:   1005.03    371.29   1000.00    328.69
# 
# ================================================================================
# |     Branch Data                                                              |
# ================================================================================
# Brnch   From   To    From Bus Injection   To Bus Injection     Loss (I^2 * Z)  
#   #     Bus    Bus    P (MW)   Q (MVAr)   P (MW)   Q (MVAr)   P (MW)   Q (MVAr)
# -----  -----  -----  --------  --------  --------  --------  --------  --------
#    1      1      2    249.77     21.60   -248.01     -4.64     1.767     17.67
#    2      1      4    186.50    -13.61   -185.44     23.58     1.063     10.63
#    3      1      5   -226.27     22.74    226.60    -22.55     0.331      3.31
#    4      2      3    -51.99    -93.97     52.12     93.39     0.125      1.25
#    5      3      4    -28.63      2.65     28.65     -3.08     0.025      0.25
#    6      4      5   -238.19     32.15    239.91    -15.66     1.716     17.16
#                                                              --------  --------
#                                                     Total:     5.027     50.27

**Selecting a proper learning model**<br>
I believe after all, these numbers are the result of some formulas. So I should look for a model that can simulate formulas. My first attemp is using neural network.<br>
I will try to simulate <code>F = m . a</code>, simply because it is the easiest physics formula I have come across.<br>
Just to make it even simpler, I use logarithms:<br>
<code>log(F) = log(m) + log(a)</code><br>
Using this method I can avoid multiplication, which is more complex than summation.

In [2]:
# importings
import numpy as np
import pandas as pd
from keras import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from random import randint
from math import log
import keras.backend as K
from keras.layers.core import Activation
from math import e

Using TensorFlow backend.


In [3]:
def populate(N, coef = 100, offset = 0):
  
  # Slow method
  # # X = np.array([])
  # # Y = np.array([])
  # # for i in range(dimension):
  # #   m = randint(0,1000) / 10
  # #   a = randint(0,1000) / 10
  # #   f = m * a
  # #   x = np.array([m, a])
  # #   y = np.array([f])
  # #   # appending
  # #   X = np.append(X, x)
  # #   Y = np.append(Y, y)
  # # return X.reshape(dimension, 2), Y.reshape(dimension, 1)

  # Much faster way:
  X = np.random.rand(N, 2) * coef + offset
  Y = np.prod(X, axis=1)
  return X, Y

In [4]:
X, Y = populate(100000)

print('X:', X.shape)
print('Y:', Y.shape)

X: (100000, 2)
Y: (100000,)


In [5]:
def activation_log(x):
  x = K.log(x)
  return x

def activation_pow(x):
  x = K.pow(e, x)
  return x

In [6]:
model = Sequential()
model.add(Activation(activation_log))
model.add(Dense(8, activation='relu', input_dim=2))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation=activation_pow))

In [7]:
model.compile(optimizer='adam', loss='mean_absolute_error')

In [8]:
model.fit(x=X, y=Y, epochs=20)

Epoch 1/20
100000/100000 [==============================] - 5s 55us/step - loss: 105.9864
Epoch 2/20
100000/100000 [==============================] - 4s 43us/step - loss: 37.0629
Epoch 3/20
100000/100000 [==============================] - 4s 44us/step - loss: 33.3059
Epoch 4/20
100000/100000 [==============================] - 5s 48us/step - loss: 32.7620
Epoch 5/20
100000/100000 [==============================] - 8s 75us/step - loss: 32.8814
Epoch 6/20
100000/100000 [==============================] - 7s 71us/step - loss: 33.2470
Epoch 7/20
100000/100000 [==============================] - 6s 62us/step - loss: 33.0393
Epoch 8/20
100000/100000 [==============================] - 7s 67us/step - loss: 34.3406
Epoch 9/20
100000/100000 [==============================] - 6s 62us/step - loss: 35.7484
Epoch 10/20
100000/100000 [==============================] - 7s 66us/step - loss: 32.6960
Epoch 11/20
100000/100000 [==============================] - 6s 64us/step - loss: 31.2375
Epoch 12/20
100000

In [9]:
# Test dataset within [0, 100]
X_test, Y_test = populate(100)

In [10]:
# Evaluation for [0, 100] range
model.evaluate(X_test, Y_test)

100/100 [==============================] - 0s 490us/step


5.949638366699219

In [11]:
# Test dataset outside of [0, 100]
X_test, Y_test = populate(100, coef=100, offset=100)

In [12]:
# Evaluation for outside of [0, 100]
model.evaluate(X_test, Y_test)

100/100 [==============================] - 0s 110us/step


86.8408203125

In [13]:
model.save(filepath='model.weights')

As it can be seen, our model doesn't perform good outside of our training data. So we should look for better options.

**Questions to ask**<br>


1.   What are the inputs?
2.   What are the outputs?
3.   Am I given the inputs or do I have to mine them from MatLab?
4.   How does MatPower calculate the output? base on what set of formula?
5.   Why don't we use MatPower instead of ML if it yields more accurate results?


